# Final Project

## Step 1: Migration Data Download

In [1]:
# Import needed Python libraries
import time
import zipfile
from getpass import getpass
from glob import glob

import pygbif.occurrences as occ
import pygbif.species as species
import requests

# Python Standard Library Packages
import os
import pathlib

# Other Packages
import earthpy # Manage local data
import pandas as pd # Work with tabular data
import geopandas as gpd # Work with geospatial vector data
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Get month names
import calendar

# Libraries for Dynamic mapping
import cartopy.crs as ccrs
import hvplot.pandas
import panel as pn


ModuleNotFoundError: No module named 'pygbif'

In [ ]:
# Create data directory
data_dir = os.path.join(

    ### Home directory
    pathlib.Path.home(),

    ### Make folders
    'earth-analytics',
    'data',

    ### Project directory for this assignment
    'final-hummbird', 
)

## Make the path
os.makedirs(data_dir, exist_ok=True)


## STEP 2: Register and log in to GBIF

In [ ]:
####--------------------------####
#### DO NOT MODIFY THIS CODE! ####
####--------------------------####
# This code ASKS for your credentials 
# and saves it for the rest of the session.
# NEVER put your credentials into your code!!!!

# GBIF needs a username, password, and email 
# All 3 need to match the account
reset = True

# Request and store username
if (not ('GBIF_USER'  in os.environ)) or reset:
    os.environ['GBIF_USER'] = input('GBIF username:')

# Securely request and store password
if (not ('GBIF_PWD'  in os.environ)) or reset:
    os.environ['GBIF_PWD'] = getpass('GBIF password:')
    
# Request and store account email address
if (not ('GBIF_EMAIL'  in os.environ)) or reset:
    os.environ['GBIF_EMAIL'] = input('GBIF email:')

## STEP 3: Get the taxon key from GBIF

One of the tricky parts about getting occurrence data from GBIF is that
species often have multiple names in different contexts. Luckily, GBIF
also provides a Name Backbone service that will translate scientific and
colloquial names into unique identifiers. GBIF calls these identifiers
**taxon keys**. 



In [ ]:
## Get the 2 distinct species key from GBIF 
backbone_ruby = species.name_backbone(name='Archilochus colubris')
backbone_broad = species.name_backbone(name='Selasphorus platycercus')

species_key_ruby = backbone_ruby['usageKey']
species_key_broad = backbone_broad['usageKey']

# Retrieve usage key and confirm existence
print("The usage key for Ruby is: " , species_key_ruby)
print("The usage key for Broad-Tail is: ", species_key_broad)

## STEP 4: Download data from GBIF

Downloading GBIF data is a multi-step process. 

For this final project, we will be using two different species of hummingbird, so will need to run this information twice.

In [ ]:
# Create data directory
ruby_dir = os.path.join(

    ### Home directory
    pathlib.Path.home(),

    ### Make folders
    'earth-analytics',
    'data',

    ### Project directory for this assignment
    'final-hummbird', 
    'ruby'
)

## Make the path
os.makedirs(ruby_dir, exist_ok=True)

In [ ]:
# Only download once
gbif_pattern_ruby = os.path.join(ruby_dir, '*.csv')

if not glob(gbif_pattern_ruby):
    
    # Only submit one request
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:
        # Submit query to GBIF
        gbif_query = occ.download([
            f"taxonKey = {species_key_ruby}",
            "hasCoordinate = True",
            f"year = 2024",
        ])
        # Take first result
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]

    # Wait for the download to build
    dld_key = os.environ['GBIF_DOWNLOAD_KEY']

    # use the occurrence command module in pygbif to get the metadata
    wait = occ.download_meta(dld_key)['status']

    # check if the status of the download = "SUCCEEDED"
    # wait and loop through until it finishes
    while not wait=='SUCCEEDED':
        wait = occ.download_meta(dld_key)['status']
        time.sleep(5)

    # Download GBIF data
    dld_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY'], 
        path=ruby_dir)
    dld_path = dld_info['path']

    # Unzip GBIF data
    with zipfile.ZipFile(dld_path) as dld_zip:
        dld_zip.extractall(path=ruby_dir)
        
    # Clean up the .zip file
    os.remove(dld_path)

    

else: print("Done!") # If file already exists in folder    

# Find the extracted .csv file path (first result)
gbif_path_ruby = glob(gbif_pattern_ruby)[0]

#Rename the file to a descriptive name
csv_filename = os.path.basename(gbif_path_ruby)

ruby_gbif = "ruby-gbif.csv"
new_path1 = os.path.join(ruby_dir, ruby_gbif)

os.rename(gbif_path_ruby, new_path1)

print(f"Renamed {csv_filename} → {ruby_gbif}")

In [ ]:
# Create data directory
broad_dir = os.path.join(

    ### Home directory
    pathlib.Path.home(),

    ### Make folders
    'earth-analytics',
    'data',

    ### Project directory for this assignment
    'final-hummbird', 
    'broad'
)

## Make the path
os.makedirs(broad_dir, exist_ok=True)

In [ ]:
# Only download once
gbif_pattern_broad = os.path.join(broad_dir, '*.csv')

if not glob(gbif_pattern_broad):
    
    # Only submit one request
    if not 'GBIF_DOWNLOAD_KEY_1' in os.environ:
        # Submit query to GBIF
        gbif_query = occ.download([
            f"taxonKey = {species_key_broad}",
            "hasCoordinate = True",
            f"year = 2024",
        ])
        # Take first result
        os.environ['GBIF_DOWNLOAD_KEY_1'] = gbif_query[0]

    # Wait for the download to build
    dld_key = os.environ['GBIF_DOWNLOAD_KEY_1']

    # use the occurrence command module in pygbif to get the metadata
    wait = occ.download_meta(dld_key)['status']

    # check if the status of the download = "SUCCEEDED"
    # wait and loop through until it finishes
    while not wait=='SUCCEEDED':
        wait = occ.download_meta(dld_key)['status']
        time.sleep(5)

    # Download GBIF data
    dld_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY_1'], 
        path=data_dir)
    dld_path = dld_info['path']

    # Unzip GBIF data
    with zipfile.ZipFile(dld_path) as dld_zip:
        dld_zip.extractall(path=broad_dir)
        
    # Clean up the .zip file
    os.remove(dld_path)

    

else: print("Done!") # If file already exists in folder    

# Find the extracted .csv file path (first result)
gbif_path_broad = glob(gbif_pattern_broad)[0]

#Rename the file to a descriptive name
csv_filename = os.path.basename(gbif_path_broad)

broad_gbif = "broad-gbif.csv"
new_path2 = os.path.join(broad_dir, broad_gbif)

os.rename(gbif_path_broad, new_path2)

print(f"Renamed {csv_filename} → {broad_gbif}")

## Step 5: Define Study Area: Ecoregions of North America and import GBIF data

In this section:
- Collect the directy for the ecoregion of North America and ensure it is plotting correctly
- Load the collected GBIF data into the notebook as a dataframe
- Change the data from a data frame (DF) to a geodatafram (GDF)

In [ ]:
### Get url for ecoregions
eco_url = ("https://storage.googleapis.com/"
       "teow2016/Ecoregions2017.zip")

### Make them machine readable
eco_dir = os.path.join(data_dir, "ecoregions")

### Make the ecoregions directory
os.makedirs(eco_dir, exist_ok = True)

### Join ecoregions shapefile path
eco_path = os.path.join(eco_dir, "ecoregions.shp")

### Download the data (once)
if not os.path.exists(eco_path):
    eco_gdf = gpd.read_file(eco_url)
    eco_gdf.to_file(eco_path)

In [ ]:
# Open up the ecoregions boundaries
eco_gdf = (
    gpd.read_file(eco_path)
    [['OBJECTID', 'ECO_NAME', 'SHAPE_AREA', 'geometry' ]]
   
)
    

# Name the index so it will match the other data later on
#eco_gdf.index.name = 'ecoregions'
# Plot the ecoregions quickly to check download
eco_gdf.plot()

In [ ]:
#View the data
eco_gdf.head()

In [ ]:
# Check that files were extracted into the correct directory
print("Files in directory after extraction:", os.listdir(data_dir))
print("Ruby folder:", os.listdir(ruby_dir))
print("Broad folder:", os.listdir(broad_dir))

## Step 6: Load GBIF dataframe and turn into geodataframes

In [ ]:
# Load the GBIF dataframe
gbif_ruby_df = pd.read_csv(
    new_path1,
    delimiter = '\t',
    index_col='gbifID',
    usecols = ['gbifID', 'decimalLatitude', 'decimalLongitude', 'month'])

### Check out data
gbif_ruby_df.head()

In [ ]:
# Load the GBIF dataframe


gbif_broad_df = pd.read_csv(
    new_path2,
    delimiter = '\t',
    index_col='gbifID',
    usecols = ['gbifID', 'decimalLatitude', 'decimalLongitude', 'month'])

### Check out data
gbif_broad_df.head()

In [ ]:
## Convert GBIF data to GDF
gbif_ruby_gdf = (
    gpd.GeoDataFrame(
        gbif_ruby_df, 
        geometry=gpd.points_from_xy(
            gbif_ruby_df.decimalLongitude, 
            gbif_ruby_df.decimalLatitude), 
        crs="EPSG:4326") #Using latitude and longitude in degrees
    # Select the desired columns
    [['month', 'geometry']]
)

# View the data
gbif_ruby_gdf

In [ ]:
## Convert GBIF data to GDF
gbif_broad_gdf = (
    gpd.GeoDataFrame(
        gbif_broad_df, 
        geometry=gpd.points_from_xy(
            gbif_broad_df.decimalLongitude, 
            gbif_broad_df.decimalLatitude), 
        crs="EPSG:4326") #Using latitude and longitude in degrees
    # Select the desired columns
    [['month', 'geometry']]
)

# View the data
gbif_broad_gdf

In [ ]:
# Confirm the data is a Geo Data Frame
gbif_ruby_gdf.info()

In [ ]:
# Confirm the data is a Geo Data Frame
gbif_broad_gdf.info()

## Step 7: Consolidate dataframe data into ecoregion data

In [ ]:
# Find out what CRS the data are in
print(eco_gdf.crs)

# Plot
eco_gdf.plot(
    color = "lightblue",
    edgecolor = "black"
)

# Give it labels
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Testing")

print("Ruby" , gbif_ruby_gdf.crs)

print(gbif_ruby_gdf)

print("Broad tailed" , gbif_broad_gdf.crs)

print(gbif_broad_gdf)

print(eco_gdf)

In [ ]:
gbif_eco_ruby_gdf = (
    eco_gdf
    # Match the CRS of the GBIF data and the ecoregions
    .to_crs(gbif_ruby_gdf.crs)
    # Find ecoregion for each observation
    .sjoin(
        gbif_ruby_gdf,
        how = 'inner', 
        
        # only include ecoregions with gbif
        predicate='contains') # Using points and polygons

    # select columns we care about
    [['OBJECTID', 'month']]
    .rename(columns = {'OBJECTID': 'eco-region'})
)
gbif_eco_ruby_gdf

In [ ]:
gbif_eco_broad_gdf = (
    eco_gdf
    # Match the CRS of the GBIF data and the ecoregions
    .to_crs(gbif_broad_gdf.crs)
    # Find ecoregion for each observation
    .sjoin(
        gbif_broad_gdf,
        how = 'inner', 
        
        # only include ecoregions with gbif
        predicate='contains') # Using points and polygons

    # select columns we care about
    [['OBJECTID', 'month']]
    .rename(columns = {'OBJECTID': 'eco-region'})
)
gbif_eco_broad_gdf

In [ ]:
gbif_eco_ruby_gdf.info()

In [ ]:
gbif_eco_broad_gdf.info()

## Step 8: Normalize hummingbird occurrences data

In [ ]:
occurrence_ruby_df = (
    gbif_eco_ruby_gdf

    # Ecoregions by month
    .groupby(['eco-region', 'month'])

    # Count the number of occurrences
    .agg(occurrences=('eco-region', 'count'))
)

# Get rid of rare observations 
occurrence_ruby_df = occurrence_ruby_df[occurrence_ruby_df.occurrences > 1] # Only include occurences wiht more than one recorded

# Take the mean by ecoregion
mean_occurrences_ruby_by_ecoregion = (
    occurrence_ruby_df
    .groupby('eco-region')
    .mean()
)
mean_occurrences_ruby_by_ecoregion

# Take the mean by month
mean_occurrences_ruby_by_month = (
    occurrence_ruby_df
    .groupby('month')
    .mean()
)
mean_occurrences_ruby_by_month

### summarize occurrences
occurrence_ruby_df = (
    gbif_eco_ruby_gdf
    # # Select only necessary columns
    # [[]]
    # For each ecoregion, for each month...
    .groupby(['eco-region', 'month'])
    # ...count the number of occurrences
    .agg(occurrences=('eco-region', 'count'))
)
print(mean_occurrences_ruby_by_ecoregion)

print(mean_occurrences_ruby_by_month)

In [ ]:
occurrence_broad_df = (
    gbif_eco_broad_gdf

    # Ecoregions by month
    .groupby(['eco-region', 'month'])

    # Count the number of occurrences
    .agg(occurrences=('eco-region', 'count'))
)

# Get rid of rare observations 
occurrence_broad_df = occurrence_broad_df[occurrence_broad_df.occurrences > 1] # Only include occurences wiht more than one recorded

# Take the mean by ecoregion
mean_occurrences_broad_by_ecoregion = (
    occurrence_broad_df
    .groupby('eco-region')
    .mean()
)
mean_occurrences_broad_by_ecoregion

# Take the mean by month
mean_occurrences_broad_by_month = (
    occurrence_broad_df
    .groupby('month')
    .mean()
)
mean_occurrences_broad_by_month

### summarize occurrences
occurrence_broad_df = (
    gbif_eco_broad_gdf
    # # Select only necessary columns
    # [[]]
    # For each ecoregion, for each month...
    .groupby(['eco-region', 'month'])
    # ...count the number of occurrences
    .agg(occurrences=('eco-region', 'count'))
)
print(mean_occurrences_broad_by_ecoregion)

print(mean_occurrences_broad_by_month)

In [ ]:
# Normalize by space and time for sampling effort
occurrence_ruby_df['norm_occurrences'] = (
    occurrence_ruby_df
    / mean_occurrences_ruby_by_month # Divide by monthly occurrences
    / mean_occurrences_ruby_by_ecoregion # Divide by ecoregion occurrences
)
occurrence_ruby_df

In [ ]:
# Normalize by space and time for sampling effort
occurrence_broad_df['norm_occurrences'] = (
    occurrence_broad_df
    / mean_occurrences_broad_by_month # Divide by monthly occurrences
    / mean_occurrences_broad_by_ecoregion # Divide by ecoregion occurrences
)
occurrence_broad_df

In [ ]:
## Look at the data

# Histogram
occurrence_ruby_df.norm_occurrences.plot.hist()

# Scatterplot
occurrence_ruby_df.reset_index().plot.scatter(
    x = 'month',
    y = 'norm_occurrences',
    c = 'eco-region',
    logy = True
)

In [ ]:
## Look at the data

# Histogram
occurrence_broad_df.norm_occurrences.plot.hist()

# Scatterplot
occurrence_broad_df.reset_index().plot.scatter(
    x = 'month',
    y = 'norm_occurrences',
    c = 'eco-region',
    logy = True
)

## Step 9: Make side by side visuals of occurrences

In [ ]:
# Create side-by-side subplots
fig, axes = plt.subplots(1, 2, figsize=(12,4)) # 1 column and 2 rows

# Plot each in their own axes
occurrence_ruby_df.norm_occurrences.plot.hist(ax=axes[0], color ="purple").set_title("Ruby Occurrences in 2024")

occurrence_broad_df.norm_occurrences.plot.hist(ax=axes[1], color = "green").set_title("Broad-Tailed Occurrences in 2024"), 

ymax = max(axes[0].get_ylim()[1], axes[1].get_ylim()[1])
axes[0].set_ylim(0, ymax)
axes[1].set_ylim(0, ymax)

plt.tight_layout()
plt.show()


In [ ]:
# Create side-by-side subplots
fig, axes = plt.subplots(1, 2, figsize=(12,4)) # 1 column and 2 rows


# Scatterplot
occurrence_ruby_df.reset_index().plot.scatter(ax=axes[0],
    x = 'month',
    y = 'norm_occurrences',
    c = 'eco-region',
    logy = True
).set_title("Ruby Occurrences in 2024")
# Scatterplot
occurrence_broad_df.reset_index().plot.scatter(ax=axes[1],
    x = 'month',
    y = 'norm_occurrences',
    c = 'eco-region').set_title("Broad-Tailed Occurrences in 2024")

ymax = max(axes[0].get_ylim()[1], axes[1].get_ylim()[1])
axes[0].set_ylim(0, ymax)
axes[1].set_ylim(0, ymax)

plt.tight_layout()
plt.show()

In [ ]:
# Simplify the geometry to speed up processing
eco_gdf.geometry = eco_gdf.simplify(0.1, preserve_topology=False)


# Change the CRS to Mercator for mapping
eco_gdf = eco_gdf.to_crs(ccrs.Mercator())

# Check that the plot runs in a reasonable amount of time
eco_gdf.hvplot(geo=True, crs=ccrs.Mercator())

In [ ]:
## Rename eco_gdf to match occurrence_df
eco_gdf = eco_gdf.rename(columns = {'OBJECTID' : 'eco-region'})

## Merge ecoregio index
eco_gdf = eco_gdf.set_index('eco-region')
eco_gdf

In [ ]:


# Join the occurrences with the plotting GeoDataFrame
occurrence_ruby_gdf = eco_gdf.join(occurrence_ruby_df)

# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = occurrence_ruby_gdf.total_bounds

# Calendar Name slider
month_widget = pn.widgets.DiscreteSlider(
    options={
        calendar.month_name[month_num]: month_num
        for month_num in range(1, 12)
        }
)




In [ ]:
occurrence_ruby_df.info()


## Step 10: Make some maps of the data!

In [ ]:
# Plot occurrence by ecoregion and month
migration_ruby_plot = (
    occurrence_ruby_gdf
    .hvplot(
      
        groupby='month',
        # Use background tiles
        geo=True, crs=ccrs.Mercator(), tiles='CartoLight',
        title="Ruby Throated Hummingbird Migration in 2024",
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=600,
        widgets={'month': month_widget},
        widget_location='bottom' # location of slider
    )
)

# Save the plot
migration_ruby_plot.save('migration-ruby-final.html', embed=True)

migration_ruby_plot.show()

In [ ]:
# Join the occurrences with the plotting GeoDataFrame
occurrence_broad_gdf = eco_gdf.join(occurrence_broad_df)

# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = occurrence_broad_gdf.total_bounds

# Calendar Name slider
month_widget = pn.widgets.DiscreteSlider(
    options={
        calendar.month_name[month_num]: month_num
        for month_num in range(1, 12)
        }
)

# Plot occurrence by ecoregion and month
migration_broad_plot = (
    occurrence_broad_gdf
    .hvplot(
        groupby='month',
        # Use background tiles
        geo=True, crs=ccrs.Mercator(), tiles='CartoLight',
        title="Broad-Tailed Hummingbird Migration in 2024",
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=600,
        widgets={'month': month_widget},
        widget_location='bottom', # location of slider,
    )
)

# Save the plot
migration_broad_plot.save('migration-broad-final.html', embed=True)

migration_broad_plot.show()

In [ ]:
# Plot occurrence by ecoregion and month


migration_ruby_broad_plot = (
    
    occurrence_ruby_gdf
    .hvplot(
        groupby='month',
        # Use background tiles
        geo=True, crs=ccrs.Mercator(), tiles='CartoLight',
        title="Broad-Tailed Hummingbird Migration in 2024",
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=600,
        widgets={'month': month_widget},
        widget_location='bottom', # location of slider,
    )

    
)

# Save the plot
#migration_ruby_broad_plot.save('migration-ruby-broad-final.html', embed=True)

migration_ruby_broad_plot.show()

In [ ]:
ruby_hv = migration_ruby_plot[0].object
broad_hv = migration_broad_plot[0].object


In [ ]:
combined = ruby_hv * broad_hv



In [ ]:
import holoviews as hv
hv.extension('bokeh')

month_dim = ruby_hv.kdims[0]  # usually 'month'

combined_dm = hv.DynamicMap(
    lambda month: (ruby_hv[month]*
                   broad_hv[month]
                  ).opts(
                      title='Ruby vs Broad-Tailed Occurrences in 2024',
                      width=800,
                      height=600
                  ),
    kdims=[month_dim]
)

combined_dm



In [ ]:
import holoviews as hv
import hvplot.pandas
import pandas as pd
hv.extension('bokeh')

def combined_layer(month):
    ruby_points = get_points_layer(ruby_hv, month).to_dataframe()
    broad_points = get_points_layer(broad_hv, month).to_dataframe()
    
    # Find overlapping geometries
    overlap = pd.merge(ruby_points, broad_points, on='geometry', how='inner')
    ruby_only = ruby_points[~ruby_points.geometry.isin(overlap.geometry)]
    broad_only = broad_points[~broad_points.geometry.isin(overlap.geometry)]
    
    # Create Points layers
    ruby_layer  = ruby_only.hvplot.points(geo=True, color='purple', alpha=0.6, size=8)
    broad_layer = broad_only.hvplot.points(geo=True, color='green', alpha=0.6, size=8)
    overlap_layer = overlap.hvplot.points(geo=True, color='red', alpha=0.8, size=8)
    
    return ruby_layer * broad_layer * overlap_layer

combined_dm = hv.DynamicMap(
    combined_layer,
    kdims=ruby_hv.kdims
)

combined_dm



## Conclusion